In [1]:
import numpy as np
import pandas as pd
import gc
from sklearn.linear_model import LinearRegression as LR

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib notebook
pd.set_option('display.max_rows', 500)

from valkyrie.avalon.feature_analyzer import FeatureMgr
from valkyrie.securities import stocks_good_dvd, parent
from valkyrie.quants import lm_fit, WinsorizedLM

/tmp/ipykernel_507730/994762019.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Load Data

In [2]:
#feature_mgr = FeatureMgr('20210101', '20210630', ['TWO PRC'] , 'latest')#
feature_mgr = FeatureMgr('20210101', '20210630', stocks_good_dvd() , 'eve_recording3')
rets = [1 , 2 , 5, 10, 12, 15]
ret_ns = [int(n * 2 * 60 * 6.5) for n in rets]

ret_col, s2_col = 'cywa_mkt_xh', 'cywa_mkt_s2h'
df = feature_mgr.calc_rets(ret_col, ret_ns, s2_col)
gc.collect()
df['parent'] = df['ticker'].apply(parent)

# PFF LM fit

In [3]:
df_pff = df[['date', 'ticker','parent'] +[c for c in df if 'cywa' in c]].copy()

In [20]:
df_pff = df[['ticker','parent'] +[c for c in df if 'cywa' in c]].copy() # + ['intercept']
ycols = [c for c in df_pff if 'cywa_mkt_xh_ret' in c and 'wgt' not in c]
wcols = ['wgt_' + c for c in ycols]

res = {}
for cywa_hl in ['short','mid','long','vlong']:#['vlong']:#['short','mid','long','vlong']:#['mkt']:#,'short','mid','long','vlong']:        
    print(f"======== { cywa_hl} ===========")
    df_pff['feature1'] = df_pff.eval(f'pff_cywa_{cywa_hl}_xh - cywa_{cywa_hl}_xh')
    df_pff['feature2'] = df_pff.eval(f'pff_cywa_mkt_xh - cywa_mkt_xh ')
                
    #xcols = f'pff_cywa_{cywa_hl}_xh cywa_{cywa_hl}_xh pff_cywa_mkt_xh cywa_mkt_xh'.split()
    xcols = ['feature1', 'feature2']
    
    df_pff['fv'] = df_pff['feature1'] * -0.442050 + df_pff['feature2'] * 0.445066
    
    res[cywa_hl] = lm_fit(df_pff.query('abs(fv) >= 0.0'), xcols, ycols, wcols)    
    display(res[cywa_hl])

======== short ===========


,cywa_mkt_xh_ret_780_n,cywa_mkt_xh_ret_1560_n,cywa_mkt_xh_ret_3900_n,cywa_mkt_xh_ret_7800_n,cywa_mkt_xh_ret_9360_n,cywa_mkt_xh_ret_11700_n
r2,0.006860,0.010898,0.028345,0.045836,0.056146,0.070549
feature1,-0.800991,-0.810924,-0.954699,-0.809137,-0.618786,-0.763009
feature2,0.802667,0.813938,0.962395,0.822219,0.634530,0.782618


======== mid ===========


,cywa_mkt_xh_ret_780_n,cywa_mkt_xh_ret_1560_n,cywa_mkt_xh_ret_3900_n,cywa_mkt_xh_ret_7800_n,cywa_mkt_xh_ret_9360_n,cywa_mkt_xh_ret_11700_n
r2,0.008378,0.011796,0.028849,0.046099,0.056313,0.070739
feature1,-0.516909,-0.543831,-0.631896,-0.596689,-0.502753,-0.600706
feature2,0.518585,0.546845,0.639593,0.609770,0.518498,0.620316


======== long ===========


,cywa_mkt_xh_ret_780_n,cywa_mkt_xh_ret_1560_n,cywa_mkt_xh_ret_3900_n,cywa_mkt_xh_ret_7800_n,cywa_mkt_xh_ret_9360_n,cywa_mkt_xh_ret_11700_n
r2,0.010384,0.012898,0.029558,0.046453,0.056514,0.070979
feature1,-0.426357,-0.442050,-0.529448,-0.500730,-0.413042,-0.498527
feature2,0.428037,0.445066,0.537149,0.513814,0.428789,0.518141


======== vlong ===========


,cywa_mkt_xh_ret_780_n,cywa_mkt_xh_ret_1560_n,cywa_mkt_xh_ret_3900_n,cywa_mkt_xh_ret_7800_n,cywa_mkt_xh_ret_9360_n,cywa_mkt_xh_ret_11700_n
r2,0.010613,0.012843,0.029624,0.046455,0.056513,0.070979
feature1,-0.349125,-0.347159,-0.429870,-0.397546,-0.326939,-0.395314
feature2,0.350813,0.350183,0.437583,0.410639,0.342696,0.414938


In [17]:
df_pff[['feature1','feature2', 'fv']].describe()

,feature1,feature2,fv
count,3.060720e+06,3.060720e+06,3.060720e+06
mean,-8.669850e-03,-8.666636e-03,-2.471804e-05
std,1.233341e-02,1.233024e-02,5.888472e-05
min,-3.401506e-02,-3.410333e-02,-8.457747e-04
25%,-1.998069e-02,-1.996441e-02,-5.879597e-05
50%,-9.841046e-03,-9.826320e-03,-2.417637e-05
75%,2.302511e-03,2.302458e-03,1.120304e-05
max,2.123742e-02,2.124449e-02,1.218583e-03


In [14]:
df_pff['fv'] = df_pff['feature1'] * -0.442050 + df_pff['feature2'] * 0.445066